In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df = application_df.drop(['EIN'], axis=1)
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [3]:
name = application_df['NAME'].value_counts()
name[name>5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: NAME, Length: 354, dtype: int64

In [4]:
replace_name = list(name[name<=5].index)
for app in replace_name:
    application_df['NAME'] = application_df['NAME'].replace(app, "Other")
application_df['NAME'].value_counts()

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [5]:
application_type_counts = application_df.groupby('APPLICATION_TYPE').size().reset_index(name='counts')
print(application_type_counts)

   APPLICATION_TYPE  counts
0               T10     528
1               T12      27
2               T13      66
3               T14       3
4               T15       2
5               T17       1
6               T19    1065
7                T2      16
8               T25       3
9               T29       2
10               T3   27037
11               T4    1542
12               T5    1173
13               T6    1216
14               T7     725
15               T8     737
16               T9     156


In [6]:
categorical_columns = application_df.select_dtypes(include=['object']).columns.tolist()
categorical_columns

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [7]:
categorical_dummies_df = pd.get_dummies(application_df)


In [10]:
X = categorical_dummies_df.drop('IS_SUCCESSFUL', axis= 1)
y = categorical_dummies_df['IS_SUCCESSFUL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=8, activation='relu', input_dim=input_features))
# Second hidden layer

nn.add(tf.keras.layers.Dense(units=4, activation='relu'))
# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 3776      
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 3817 (14.91 KB)
Trainable params: 3817 (14.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5418 - accuracy: 0.7452
Epoch 2/100
858/858 [==============================] - 1s 1ms/step - loss: 0.4539 - accuracy: 0.7825
Epoch 3/100
858/858 [==============================] - 1s 1ms/step - loss: 0.4420 - accuracy: 0.7871
Epoch 4/100
858/858 [==============================] - 1s 1ms/step - loss: 0.4373 - accuracy: 0.7918
Epoch 5/100
858/858 [==============================] - 1s 1ms/step - loss: 0.4341 - accuracy: 0.7918
Epoch 6/100
858/858 [==============================] - 1s 1ms/step - loss: 0.4284 - accuracy: 0.7934
Epoch 7/100
858/858 [==============================] - 1s 1ms/step - loss: 0.4270 - accuracy: 0.7933
Epoch 8/100
858/858 [==============================] - 1s 1ms/step - loss: 0.4257 - accuracy: 0.7946
Epoch 9/100
858/858 [==============================] - 1s 1ms/step - loss: 0.4248 - accuracy: 0.7941
Epoch 10/100
858/858 [==============================] - 1s 1ms/step - loss: 0.4235 - accura

In [15]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.4604 - accuracy: 0.7834 - 459ms/epoch - 2ms/step
Loss: 0.46039384603500366, Accuracy: 0.7833819389343262


In [16]:
nn.save("AlphabetSoupCharity_Optimization.h5")

/Users/fizatariq/anaconda3/envs/dev3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
